# 如何使用千帆 Python SDK 搭配预置大模型服务进行批量推理

在 0.2.4 中，千帆 Python SDK 增加了对批量推理的支持，使用该功能需要视情况开通千帆的模型服务，确保您的账号可以调用您想进行批量推理的服务。

# 准备工作

在开始之前，请确保你的千帆 Python SDK 已经升级到了 0.2.4 及以上版本。

In [ ]:
pip install -U "qianfan>=0.2.4"

并且在环境变量中设置好 Access Key 与 Secret Key

In [ ]:
import os

os.environ['QIANFAN_ACCESS_KEY'] = 'your_access_key'
os.environ['QIANFAN_SECRET_KEY'] = 'your_secret_key'
os.environ['QIANFAN_CONSOLE_API_RETRY_COUNT'] = "10"
os.environ['QIANFAN_CONSOLE_API_RETRY_TIMEOUT'] = "60"

# 正文

为了开始批量推理，我们首先需要获取到用于做批量推理输入的数据集文件，并且指定用做推理输入的列名

In [ ]:
from qianfan.dataset import Dataset

dataset_file_path = "your_path"
dataset_input_column_list = ["column1", "column2"]

ds = Dataset.load(data_file=dataset_file_path, input_columns=dataset_input_column_list)

# 如果用户需要使用对话类数据集进行批量推理，还需要指定应用列
# reference_column = "column3"
# ds = Dataset.load(data_file=dataset_file_path, input_columns=dataset_input_column_list, reference_column=reference_column)

# 预览数据格式
print(ds.list(0))

在导入之后，用户可以根据自己的需求，传入不同的参数来使用不同的方式进行推理

In [ ]:
# 用户可以设置 service_model 为自己想要的模型名，来直接对数据进行批量推理，以 EB 4 为例
result = ds.test_using_llm(service_model="ERNIE-Bot-4")

# 用户还可以设置 service_endpoint 来使用预置或自己的服务。
result = ds.test_using_llm(service_endpoint="/chat/completions_pro")

# 如果是使用对话类数据集进行批量推理，需要设置 is_chat_service=True
result = ds.test_using_llm(service_model="ERNIE-Bot-4", is_chat_service=True)

拿到的 `result` 对象也是一个 `Dataset` 对象，可以继续使用千帆 Python SDK 进行后续处理，或者直接保存到本地。

In [ ]:
print(result.list(0))
      
dataset_save_file_path = "your_path"

result.save(data_file=dataset_save_file_path)

# Prompt 模板和人设参数

在调用 `test_using_llm` 时，用户还可以传入一些额外参数来支持额外的功能，比如 Prompt 模板和人设参数。

当进行的是非对话类推理时，用户可以传入 `prompt_template` 参数来传递一个 Prompt 模板。`prompt_template` 是一个千帆 Python SDK 的 `Prompt` 对象，用户可以通过设置 `Prompt` 对象的 `template` 成员来自定义被用于推理的模板，模板渲染出来的内容将会被作为最终输入提交给大模型。假设我们有一个数据集，其中的 "question" 与 "answer" 将会被用作大模型的输入，那么我们就可以这么指定模板：

In [ ]:
from qianfan.components import Prompt

prompt = Prompt(template="这个是问题:{question}, 这个是答案:{answer}")

# 传递给函数
result = ds.test_using_llm(service_model="ERNIE-Bot-4", prompt_template=prompt)


当进行的是对话类推理时，用户可以传入 `system_prompt` 参数来指定对话中大模型需要遵守的人设

In [ ]:
result = ds.test_using_llm(service_model="ERNIE-Bot-4", system_prompt="人设 prompt")